In [39]:
import pandas as pd
import ast
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [51]:
df = pd.read_csv("/content/restaurants_data_merged_final.csv")
df['cuisine'] = df['cuisine'].apply(ast.literal_eval)
df

,name,price,rating,review_count,cuisine,address,display_phone,comments
0,123 Burger Shot Beer,1,3.0,1000.0,"[american, sportsbars, tradamerican, chicken_w...","738 10th Ave, Hells Kitchen, NY 10019",(212) 315-0123,NaN
1,One Stop Patty Shop,1,4.0,40.0,"[bakery, caribbean, breakfast_brunch]","1708 Amsterdam Ave, Harlem, NY 10031",(212) 491-7466,NaN
2,108 Food Dried Hot Pot,2,3.5,139.0,"[chinese, hotpot]","2794 Broadway, East Harlem, NY 10025",(917) 675-6878,NaN
3,Cookshop,2,4.0,1000.0,"[american, newamerican, breakfast_brunch, wine...","156 10th Ave, Midtown West, NY 10011",(212) 924-4440,NaN
4,11 Hanover Greek,3,4.0,122.0,"[greek, seafood, wine_bars]","11 Hanover Sq, Tribeca, NY 10005",(212) 785-4000,NaN
...,...,...,...,...,...,...,...,...
4593,ATRIO Wine Bar & Restaurant,2,5.0,215.0,[contemporary_american],"102 North End Avenue New York, NY 10282",NaN,NaN
4594,Bobby Van's Steakhouse - Broad Street,2,4.0,768.0,[steakhouse],"25 Broad Street New York, NY 10004",NaN,NaN
4595,Brasserie Seoul,2,4.0,20.0,[contemporary_french],"300 Schermerhorn St. Brooklyn, NY 11217",NaN,NaN
4596,Greenhouse Cafe,2,5.0,86.0,[continental],"7717 3rd Avenue Brooklyn, NY 11209-3001",NaN,NaN


In [43]:
columns_wanted = list(df.columns)
columns_wanted.remove('cuisine')
columns_wanted.remove('display_phone')
columns_wanted.remove('comments')
columns_wanted.remove('address')
columns_wanted.remove('review_count')
columns_wanted.remove('name')
columns_wanted

['price', 'rating']

In [44]:
#get one hot encodings for restaurant categories
one_hot = pd.get_dummies(df['cuisine'].explode()).groupby(level=0).sum()

In [48]:
#normalize ratings and combine data with one hot encoding
newdf = df.loc[:, columns_wanted].copy()

scaler = StandardScaler()
newdf['rating'] = scaler.fit_transform(newdf[['rating']])

newdf = pd.concat([newdf, one_hot], axis = 1)
newdf

,price,rating,acaibowls,afghan,afghani,african,american,arabian,arcades,argentine,...,venezuelan,venues,vietnamese,vietnamese/cambodian/malaysia,waffles,wedding_planning,whiskeybars,wine_bar,wine_bars,wraps
0,1,-1.718343,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.093799,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,-0.812272,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,0.093799,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,3,0.093799,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4593,2,1.905942,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4594,2,0.093799,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4595,2,0.093799,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4596,2,1.905942,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
#Compute cosine similarity matrix
sim_matrix = cosine_similarity(newdf)
sim_df = pd.DataFrame(sim_matrix, index = df['name'], columns = df['name'])
sim_df.round(3)

name,123 Burger Shot Beer,One Stop Patty Shop,108 Food Dried Hot Pot,Cookshop,11 Hanover Greek,118 Kitchen,123BSB,Mondrian Terrace,16 Handles,1803,...,Bar Freud,Yellow Magnolia Café,Benchmark Restaurant,Altesi Downtown,Harvey @ The Williamsburg Hotel,ATRIO Wine Bar & Restaurant,Bobby Van's Steakhouse - Broad Street,Brasserie Seoul,Greenhouse Cafe,"Lorenzo's Restaurant, Bar & Caberet - Hilton Garden Inn - SI"
name,,,,,,,,,,,,,,,,,,,,,
123 Burger Shot Beer,1.000,0.149,0.467,0.356,0.290,0.171,0.894,0.230,0.133,0.152,...,0.291,0.450,-0.154,-0.154,0.291,-0.154,0.291,0.291,-0.154,-0.154
One Stop Patty Shop,0.149,1.000,0.372,0.531,0.434,0.290,0.291,0.355,0.225,0.349,...,0.448,0.448,0.370,0.370,0.448,0.370,0.448,0.448,0.370,0.370
108 Food Dried Hot Pot,0.467,0.372,1.000,0.537,0.662,0.430,0.663,0.537,0.333,0.412,...,0.679,0.679,0.323,0.323,0.679,0.323,0.679,0.679,0.323,0.323
Cookshop,0.356,0.531,0.537,1.000,0.715,0.409,0.542,0.501,0.317,0.600,...,0.633,0.791,0.503,0.503,0.633,0.503,0.633,0.633,0.503,0.503
11 Hanover Greek,0.290,0.434,0.662,0.715,1.000,0.501,0.534,0.613,0.388,0.682,...,0.775,0.775,0.607,0.607,0.775,0.607,0.775,0.775,0.607,0.607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ATRIO Wine Bar & Restaurant,-0.154,0.370,0.323,0.503,0.607,0.428,0.078,0.503,0.331,0.670,...,0.635,0.635,1.000,0.884,0.788,1.000,0.635,0.635,0.884,0.884
Bobby Van's Steakhouse - Broad Street,0.291,0.448,0.679,0.633,0.775,0.517,0.544,0.633,0.401,0.610,...,0.800,0.800,0.635,0.635,0.800,0.635,1.000,0.800,0.635,0.635
Brasserie Seoul,0.291,0.448,0.679,0.633,0.775,0.517,0.544,0.633,0.401,0.610,...,0.800,0.800,0.635,0.635,0.800,0.635,0.800,1.000,0.635,0.635


In [56]:
def recommend(restaurant_name, df, sim_matrix, top_k=3):
    idx = df[df['name'] == restaurant_name].index[0]
    scores = list(enumerate(sim_matrix[idx]))

    # sort by similarity score, exclude the restaurant itself
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    scores = [(i, s) for i, s in scores if i != idx]

    top = scores[:top_k]
    results = df.iloc[[i for i, _ in top]][['name', 'cuisine', 'price', 'rating']].copy()
    results['similarity'] = [round(s, 3) for _, s in top]
    return results.reset_index(drop=True)

print(recommend('Cookshop', df, sim_matrix))

                  name                                            cuisine  \
0  Murray's Cheese Bar  [american, newamerican, wine_bars, breakfast_b...   
1            BLACKBARN  [american, newamerican, breakfast_brunch, wine...   
2    Elephant & Castle          [american, newamerican, breakfast_brunch]   

   price  rating  similarity  
0      2     4.0       1.000  
1      3     4.0       0.981  
2      2     4.0       0.935  
